<a href="https://colab.research.google.com/github/Muhammad-Awais-Awan/TensorFLow-MachineLearning/blob/main/IrisClassification%20using%20DNN%20classifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [6]:
import tensorflow as tf
import pandas as pd


**DATASET**


*   Setosa
*   Versicolor
*   Virginica

The information about each flower is the following



*   sepal length
*   sepal width
*   petal length
*   petal width











In [7]:
CSV_COLUMN_NAMES=['SepalLength','SepalWidth','PetalLength','PetalWidth','Species']
SPECIES=['Setosa','Versicolor','Virginica']
#lets define some constants to help us later


In [9]:
train_path=tf.keras.utils.get_file(
    "iris_training.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_training.csv"
    )
test_path=tf.keras.utils.get_file(
    "iris_test.csv","https://storage.googleapis.com/download.tensorflow.org/data/iris_test.csv"
)

train=pd.read_csv(train_path,names=CSV_COLUMN_NAMES,header=0)
test=pd.read_csv(test_path,names=CSV_COLUMN_NAMES,header=0)
# Here we use keras (a module inside of TensorFlow) to grab our datasets and read them into a pandas dataframe

In [10]:
train.head()

,SepalLength,SepalWidth,PetalLength,PetalWidth,Species
0,6.4,2.8,5.6,2.2,2
1,5.0,2.3,3.3,1.0,1
2,4.9,2.5,4.5,1.7,2
3,4.9,3.1,1.5,0.1,0
4,5.7,3.8,1.7,0.3,0


In [11]:
train_y=train.pop('Species')
test_y=test.pop('Species')
train.head() # the species column is now gone



,SepalLength,SepalWidth,PetalLength,PetalWidth
0,6.4,2.8,5.6,2.2
1,5.0,2.3,3.3,1.0
2,4.9,2.5,4.5,1.7
3,4.9,3.1,1.5,0.1
4,5.7,3.8,1.7,0.3


In [12]:
train.shape # we have 120 enteries with 4 features

(120, 4)

In [13]:
train_y.head()

0    2
1    1
2    2
3    0
4    0
Name: Species, dtype: int64

In [14]:
def input_fn(features,labels, training=True,batch_size=256):
  #Convert the inputs to a Dataset.
  dataset=tf.data.Dataset.from_tensor_slices((dict(features),labels))
  #Shuffle and repeat if you are in training mode.
  if training:
    dataset= dataset.shuffle(1000).repeat()
  return dataset.batch(batch_size)  

In [15]:
#feature columns describes how to use the input
my_feature_columns=[]
for key in train.keys():
  my_feature_columns.append(tf.feature_column.numeric_column(key=key))
print(my_feature_columns)  


[NumericColumn(key='SepalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='SepalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalLength', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None), NumericColumn(key='PetalWidth', shape=(1,), default_value=None, dtype=tf.float32, normalizer_fn=None)]


BUILDING THE MODEL

There are variety of different estimators/models that we can pick from. some options are listed below.


*   DNN Classifier( Deep Neural Network)
*   Linear Classifier 



In [18]:
# Build a DNN with 2 hidden layers with 30 and 10 hidden nodes each
classifier =tf.estimator.DNNClassifier(
    feature_columns=my_feature_columns,
    #Two Hidden Layers of 30 and 10 nodes respectively.
    hidden_units=[30,10],
    #The model must choose between 3 classes.
    n_classes=3)

In [20]:
classifier.train(
    input_fn=lambda:input_fn(train,train_y,training=True),#lambda is use to define one line functions
    steps=5000
)

Instructions for updating:
Use standard file utilities to get mtimes.


In [24]:

eval_result=classifier.evaluate(input_fn=lambda:input_fn(test,test_y,training=False))
print('\nTest set Accuracy : {accuracy:0.3f}\n'.format(**eval_result))


Test set Accuracy : 0.933



In [31]:
def input_fn(features, batch_size=256):
  #convert the inputs to a Dataset without labels.
  return tf.data.Dataset.from_tensor_slices(dict(features)).batch(batch_size)
features=['SepalLength','SepalWidth','PetalLength','PetalWidth']
predict={}
print("Please type numeric values as prompted.")
for feature in features:
  valid=True
  while valid:
    val=input(feature + ":")
    if not val.isdigit():valid=False
  predict[feature]=[float(val)]
predictions= classifier.predict(input_fn=lambda:input_fn(predict)) 
for pred_dict in predictions:
  print(pred_dict)
  class_id= pred_dict['class_ids'][0]
  probability=pred_dict['probabilities'][class_id]

  print('Prediction is "{}" ({: 1f}%)'.format(SPECIES[class_id],100*probability))     

Please type numeric values as prompted.
SepalLength:5.1
SepalWidth:3.3
PetalLength:1.7
PetalWidth:0.5
{'logits': array([-0.43832183, -1.9214184 , -3.3312786 ], dtype=float32), 'probabilities': array([0.77982074, 0.17696775, 0.04321153], dtype=float32), 'class_ids': array([0]), 'classes': array([b'0'], dtype=object), 'all_class_ids': array([0, 1, 2], dtype=int32), 'all_classes': array([b'0', b'1', b'2'], dtype=object)}
Prediction is "Setosa" ( 77.982074%)
